### Trabalho 2 
#### Igor Loredo - 11275071
#### Natã Botelho - 11275105 
#### A partir do material visto nas aulas, criar portfólios com maior rentabilidade e outro com menor volatilidade 


In [ ]:
import pandas.testing as tm
from scipy.optimize import minimize
import statsmodels.api as sm
import numpy as np
import pandas as pd
from operator import itemgetter

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm




```
Downloading Dados
```



In [ ]:
data_inicial = 13 #  13 = 01/2005
data_final   = data_inicial + 192 # 12/2020
colunas = 291 #len(comp_indice.columns)
step_port = 1
step_eval = 1


#Composição do índice IBX
comp_indice=pd.read_excel('Dados-Comp-IBRX.xlsx', engine='openpyxl')
comp_indice.set_index(keys = 'Data', inplace = True)

#Preços de fechamento dos ativos
fechamento=pd.read_excel('Dados-Fechamento.xlsx', engine='openpyxl')
fechamento.set_index(keys = 'Data', inplace = True)

#Indices de referência (Ibov, IBX, SELIC...)
referencias=pd.read_excel('Dados-Base.xlsx', engine='openpyxl')
referencias.set_index(keys = 'Data', inplace = True)
print(referencias)
#-------------------------------------------

# Fator Qualidade (ROIC das empresas)
fator_ROIC=pd.read_excel('Dados-ROIC-A2.xlsx', engine='openpyxl')
fator_ROIC.set_index(keys = 'Data', inplace = True)
ranked_ROIC=fator_ROIC.rank(axis=1, numeric_only=True, ascending=False, method='first')

# Fator Momentum (Momentum de 12 meses)
fator_Mom=pd.read_excel('Dados-Momentum-12.xlsx', engine='openpyxl')
fator_Mom.set_index(keys = 'Data', inplace = True)
ranked_Mom=fator_Mom.rank(axis=1, numeric_only=True, ascending=False, method='first')

#Fator Tamanho (Valor de mercado das empresas)
fator_Val_Merc=pd.read_excel('Dados-Val-Merc.xlsx', engine='openpyxl')
fator_Val_Merc.set_index(keys = 'Data', inplace = True)
ranked_Val_Merc=fator_Val_Merc.rank(axis=1, numeric_only=True, ascending=True, method='first')

#Fator Valor (Preço / Valor Patrimonial)
fator_PVP=pd.read_excel('Dados-PVP.xlsx', engine='openpyxl')
fator_PVP.set_index(keys = 'Data', inplace = True)
ranked_PVP=fator_PVP.rank(axis=1, numeric_only=True, ascending=True, method='first')

#Fator Volatilidade (Volatilidade em 12 meses)
fator_Vol=pd.read_excel('Dados-Vol-12.xlsx', engine='openpyxl')
fator_Vol.set_index(keys = 'Data', inplace = True)
ranked_Vol=fator_Vol.rank(axis=1, numeric_only=True, ascending=True, method='first')

print("Periodo de avaliacao - de:", comp_indice.index[data_inicial], "(", data_inicial, ")",  "ate:", comp_indice.index[data_final-1], "(", data_final-1, ")")
print("Rebalanceamento a cada", step_eval,"/", step_port, "meses")

               IBOV      IBXX        SELIC       IPCA
Data                                                 
Dez-2003   22236.00   6004.00  1031.614995  10.642263
Jan-2004   21851.00   5875.00  1044.691245  10.723145
Fev-2004   21755.00   6041.00  1056.019816  10.788556
Mar-2004   22142.00   6077.00  1070.583633  10.839262
Abr-2004   19607.00   5283.00  1083.236327  10.879367
...             ...       ...          ...        ...
Ago-2021  118781.03  50920.04  6090.183000  28.050375
Set-2021  110979.10  47360.36  6117.101080  28.375759
Out-2021  103500.71  44133.83  6146.829470  28.730456
Nov-2021  101915.45  43387.81  6182.896280  29.003396
Dez-2021  104822.44  44749.56  6230.448260  29.215120

[217 rows x 4 columns]
Periodo de avaliacao - de: Jan-2005 ( 13 ) ate: Dez-2020 ( 204 )
Rebalanceamento a cada 1 / 1 meses


In [ ]:
#Avaliação de um portfólio. 
#Parâmetros: (portfólio, histórico de preços dos ativos)
#Retorno: vetor com retorno acumulado, vetor com retornos periódicos, vetor com drawdown, retorno anualizado, volatilidade anualizada

def EvalPort(port, fechamento):
    port_acc_vet = []
    port_chg_vet = []
    port_ddown_vet = []
    
    port_acc = 1.0
    port_acc_vet.append(1.0)
    cost_trans = 0.0006
    #cost_trans = 0.0005 + (0.004*step_eval/12)
    
    for lin in range(data_inicial, data_final, step_eval):
        cont = 0.0
        rent = 0.0
        for col in range(0, colunas):
            if (port.iat[lin-1, col] > 0 and fechamento.iat[lin-1, col]>0 and fechamento.iat[lin-1+step_eval, col]>0):
                rent = rent + (fechamento.iat[lin-1+step_eval,col]/fechamento.iat[lin-1,col]-1)*(port.iat[lin-1, col])
                cont = cont + port.iat[lin-1, col]
        # print(cont)
        # if (cont == 0):
        #   return [1,1], [1,1], [0,0], 0, 0.000001
        if cont != 0:
          port_acc = port_acc * (1.0 + rent/cont - cost_trans)
          port_chg_vet.append(rent/cont - cost_trans)
          port_acc_vet.append(port_acc)
          port_ddown_vet.append(port_acc/(np.max(port_acc_vet))-1)
    
    ret_aa = pow(port_acc, 12/(data_final-data_inicial))-1
    vol_aa = np.std(port_chg_vet)*((12/step_eval)**(1/2))
    return port_acc_vet, port_chg_vet, port_ddown_vet, ret_aa, vol_aa

 portfolio 




In [ ]:
def SelPort2Par(ranked_1, param_1, ranked_2, param_2):
  port_ranked_final = ranked_1.copy()
  port_ranked_final.loc[:, :] = 0
  port_counter = 0
  port_final_list = []
  cols_sum = []
  for lin in range(data_inicial, data_final, step_port):
    for col in range(0, colunas):
      if(ranked_1.iat[lin-1, col]  >= 1 and (ranked_1.iat[lin-1, col]  <= param_1)
        and (ranked_2.iat[lin-1, col]  >= 1) and (ranked_2.iat[lin-1, col]  <= param_2)):

        entry = {'lin':lin-1, 'col':col, 'rank1':ranked_1.iat[lin-1, col], 'rank2':ranked_2.iat[lin-1, col]}
        port_final_list.append(entry)
        
  port_final_list = sorted(port_final_list, key=itemgetter('col'))

  entries_counter_list = []

  for entry in port_final_list:
    entries_per_col = len(list(filter(lambda col_entry: col_entry['col'] == entry['col'], port_final_list)))
    if len(list(filter(lambda col_entry: col_entry['col'] == entry['col'], entries_counter_list))) == 0:
      entry_counter = {'col': entry['col'], 'counter': entries_per_col}
      entries_counter_list.append(entry_counter)
    
  entries_counter_list = sorted(entries_counter_list, key=itemgetter('counter'), reverse=True)

  for entry_counter in entries_counter_list:
    if port_counter < 10:
      for item in port_final_list:
        if item['col'] == entry_counter['col']:
          port_ranked_final.iat[item['lin'], item['col']] = 1
      port_counter += 1
  return port_ranked_final

**Rankings dos fatores:**

*   Momentum (Retorno 12 meses):     ranked_Mom
*   Qualidade (ROIC):    ranked_ROIC
*   Tamanho (Valor Patrimonial):      ranked_Val_Merc
*   Valor (Preço/Val. Pat.):        ranked_PVP
*   Volatilidade (Volatilidade 12 meses): ranked_Vol


In [ ]:
# momentum e roic para trazer melhor rentabilidade

ranked2 = SelPort2Par(ranked_Mom, 40, ranked_ROIC, 50)

port_acc_vet2, port_chg_vet2, port_ddown_vet2, ret_aa2, vol_aa2 = EvalPort(ranked2, fechamento)
print("\nPort Fator:\nRet. Acc.:",round(port_acc_vet2[-1]*100-100, 2) ,"% Ret. Anual.:",round(ret_aa2*100,2), "% Vol.:", round(vol_aa2*100,2), "% Ret/Vol:", round(ret_aa2/vol_aa2, 2), "DDown:", round(np.min(port_ddown_vet2)*100,2), "%")



Port Fator:
Ret. Acc.: 2358.61 % Ret. Anual.: 22.16 % Vol.: 23.54 % Ret/Vol: 0.94 DDown: -40.77 %


In [ ]:
#momentum e volatilidade para menor volatilidade
ranked2 = SelPort2Par(ranked_Mom, 40, ranked_Vol, 50)

port_acc_vet2, port_chg_vet2, port_ddown_vet2, ret_aa2, vol_aa2 = EvalPort(ranked2, fechamento)
print("\nPort Fator:\nRet. Acc.:",round(port_acc_vet2[-1]*100-100, 2) ,"% Ret. Anual.:",round(ret_aa2*100,2), "% Vol.:", round(vol_aa2*100,2), "% Ret/Vol:", round(ret_aa2/vol_aa2, 2), "DDown:", round(np.min(port_ddown_vet2)*100,2), "%")



Port Fator:
Ret. Acc.: 1674.09 % Ret. Anual.: 19.69 % Vol.: 23.78 % Ret/Vol: 0.83 DDown: -46.25 %
